In [1]:
import cifar10, cifar10_input
import tensorflow as tf
import numpy as np
import time

/home/hhl/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
FLAGS = tf.app.flags.FLAGS

定义batch_size 迭代次数 和cifar-10的数据默认路径

In [3]:
max_steps = 3000
batch_size = 128
data_dir = '/home/hhl/machine_learning_tutorial/tensorflow_tutorial/advance_cnn/data'

 * 准备数据

maybe_download_and_extract() 报错
UnrecognizedFlagError: Unknown command line flag 'f'

解决方案 tf.app.flags.DEFINE_string('f', '', 'kernel')

In [4]:
# tf.app.flags.DEFINE_string('f', '', 'kernel')

In [5]:
# cifar10.maybe_download_and_extract()

In [6]:
images_train, labels_train = cifar10_input.distorted_inputs(data_dir = data_dir,batch_size = batch_size)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [7]:
image_test,labels_test = cifar10_input.inputs(eval_data = True, data_dir = data_dir, batch_size = batch_size)

* var 是正常的参数，l2_loss 是将var的正则未来要放到loss中，w1是该正则的系数。

In [8]:
def variable_with_weigth_loss(shape, stddev, w1):
    var = tf.Variable(tf.truncated_normal(shape, stddev = stddev))
    if w1 is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var), w1, name = 'weight_loss')
        tf.add_to_collection('losses', weight_loss)
    return var

In [9]:
image_holder = tf.placeholder(tf.float32, [batch_size, 24,24, 3])
label_holder = tf.placeholder(tf.float32, [batch_size,])

 * 第一层卷积

In [10]:
weight1 = variable_with_weigth_loss(shape = [5, 5, 3, 64], stddev = 0.05, w1 = 0.0)
kernel1 = tf.nn.conv2d(image_holder, weight1, [1, 1, 1, 1], padding = 'SAME')
bias1 = tf.Variable(tf.constant(0.0, shape = [64]))
conv1 = tf.nn.relu(tf.nn.bias_add(kernel1,bias1))
pool1 = tf.nn.max_pool(conv1, ksize = [1, 3, 3, 1], strides = [1, 2, 2, 1], padding='SAME')
norm1 = tf.nn.lrn(pool1, 4, bias = 1.0, alpha = 0.001 / 9.0, beta = 0.75)

* 第二层卷积

In [11]:
weight2 = variable_with_weigth_loss(shape = [5, 5, 64, 64], stddev = 0.05, w1 = 0.0)
kernel2 = tf.nn.conv2d(norm1, weight2, [1, 1, 1, 1], padding = 'SAME')
bias2 = tf.Variable(tf.constant(0.1, shape = [64]))
conv2 = tf.nn.relu(tf.nn.bias_add(kernel2, bias2))
norm2 = tf.nn.lrn(conv2, 4, bias = 1.0, alpha=0.001/9.9,beta = 0.75)
pool2 = tf.nn.max_pool(norm2, ksize = [1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

* 全链接层

In [12]:
reshape = tf.reshape(pool2, [batch_size, -1])
dim = reshape.get_shape()[1].value
weight3 = variable_with_weigth_loss(shape=[dim,384], stddev=0.04, w1=0.004)
bias3 = tf.Variable(tf.constant(0.1, shape=[384]))
local3 = tf.nn.relu(tf.matmul(reshape, weight3) + bias3)

* 第二全链接层

In [13]:
weight4 = variable_with_weigth_loss(shape = [384,192], stddev = 0.04, w1 = 0.004)
bias4 = tf.Variable(tf.constant(0.1, shape = [192]))
local4 = tf.nn.relu(tf.matmul(local3, weight4) + bias4)

* 输出

In [14]:
weight5 = variable_with_weigth_loss(shape = [192, 10], stddev=1/192.0, w1 = 0.0)
bias5 = tf.Variable(tf.constant(0.0, shape=[10]))
logits = tf.add(tf.matmul(local4, weight5),bias5)

 * 以上即使inference 的部分，即正向传播的部分。
 
 下面开始计算loss

In [15]:
def loss(logits,labels):
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, 
                                                                   labels = labels, 
                                                                   name = 'cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name = 'cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)
    return tf.add_n(tf.get_collection('losses'), name = 'total_loss')

In [16]:
loss = loss(logits,label_holder)

In [17]:
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [18]:
# top_k_op = tf.nn.in_top_k(logits, label_holder, 1)

In [19]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [20]:
tf.train.start_queue_runners()

[<Thread(QueueRunnerThread-input_producer-input_producer/input_producer_EnqueueMany, started daemon 139741164979968)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139741156587264)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139741148194560)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139741139801856)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139740779112192)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139740770719488)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139740762326784)>,
 <Thread(QueueRunnerThread-shuffle_batch/

In [23]:
for step in (range(max_steps)):
    start_time = time.time()
    image_batch, label_batch = sess.run([images_train, labels_train])
    _, loss_value = sess.run([train_op, loss], feed_dict = {image_holder:image_batch, label_holder:label_batch})
    duration = time.time() - start_time
    if step % 10 == 0:
        example_per_sec = batch_size / duration
        sec_per_batch = float(duration)
        format_str = ('step %d, loss = %.2f ( %.1f example/sec; %.3f sec/batch)')
        print(format_str % (step,loss_value, example_per_sec,sec_per_batch))

ResourceExhaustedError: OOM when allocating tensor with shape[128,64,24,24] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: gradients/MaxPool_grad/MaxPoolGrad = MaxPoolGrad[T=DT_FLOAT, data_format="NHWC", ksize=[1, 3, 3, 1], padding="SAME", strides=[1, 2, 2, 1], _device="/job:localhost/replica:0/task:0/device:GPU:0"](Relu, MaxPool, gradients/LRN_grad/LRNGrad)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'gradients/MaxPool_grad/MaxPoolGrad', defined at:
  File "/home/hhl/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/hhl/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/hhl/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/hhl/anacon